# Data Validation

This notebook explores the historical data given in a spreadsheet to make sure that:

1. Exercises only list muscle groups that actually exist
1. Workout routines only list exercises that already exist
1. Workouts only list routines that aleady exist and only list exercises given in those routines

In [62]:
from pathlib import Path

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [63]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
DATA_DIR = Path("../resources")

In [64]:
client = BenchmarkClient(url=URL)

In [65]:
df_muscle_groups = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="musclegroups")
df_muscle_groups.head()

,name
0,chest
1,triceps
2,biceps
3,shoulders
4,quadriceps


In [66]:
df_exercises = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="exercises")
df_exercises["musclegroups"] = df_exercises["musclegroups"].str.split(",")
df_exercises = df_exercises.explode("musclegroups")
df_exercises["musclegroups"] = df_exercises["musclegroups"].str.strip()
df_exercises.head()

,name,musclegroups
0,Bench Press,chest
0,Bench Press,triceps
1,Lean-In Cable Row,back
1,Lean-In Cable Row,biceps
2,Dumbbell Shoulder Press Neutral Grip,shoulders


In [67]:
outer = df_exercises.merge(
    df_muscle_groups,
    how="outer",
    left_on="musclegroups",
    right_on="name",
    indicator=True,
)
if len(outer._merge.unique()) > 1:
    print("oh no")

In [68]:
df_routines = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workoutroutines")
df_routines["exercise"] = df_routines["exercise"].str.lower()
df_exercises["name"] = df_exercises["name"].str.lower()
df_routines.head()

,name,exercise,reps
0,Wave Loading 321 Lower 1,trap bar deadlift,3
1,Wave Loading 321 Lower 1,trap bar deadlift,2
2,Wave Loading 321 Lower 1,trap bar deadlift,1
3,Wave Loading 321 Lower 1,trap bar deadlift,3
4,Wave Loading 321 Lower 1,trap bar deadlift,2


In [69]:
outer = df_routines.merge(
    df_exercises, how="outer", left_on="exercise", right_on="name", indicator=True
)
if len(outer[outer["_merge"] == "left_only"]):
    print("oh no")

In [70]:
df_workouts = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df_workouts["exercise"] = df_workouts["exercise"].str.lower()
df_workouts.head()

,workoutroutine,date,exercise,weight,reps
0,Hypertrophy Legs,2023-04-22,lying leg curls,35.0,16
1,Hypertrophy Legs,2023-04-22,lying leg curls,35.0,16
2,Hypertrophy Legs,2023-04-22,lying leg curls,35.0,10
3,Hypertrophy Legs,2023-04-22,hack squat,60.0,8
4,Hypertrophy Legs,2023-04-22,hack squat,60.0,6


In [71]:
outer = df_workouts.merge(
    df_routines, how="outer", left_on="workoutroutine", right_on="name", indicator=True
)
if len(outer[outer["_merge"] == "left_only"]):
    print("oh no")

In [72]:
workout_dates = df_workouts.date.unique()
df_workouts = df_workouts.set_index("date")
df_workouts

,workoutroutine,exercise,weight,reps
date,,,,
2023-04-22,Hypertrophy Legs,lying leg curls,35.0,16
2023-04-22,Hypertrophy Legs,lying leg curls,35.0,16
2023-04-22,Hypertrophy Legs,lying leg curls,35.0,10
2023-04-22,Hypertrophy Legs,hack squat,60.0,8
2023-04-22,Hypertrophy Legs,hack squat,60.0,6
...,...,...,...,...
2024-10-22,Upper Lower 3 Sets Upper 2,dumbbell lateral raises,7.0,20
2024-10-22,Upper Lower 3 Sets Upper 2,ez-bar curls,17.5,18
2024-10-22,Upper Lower 3 Sets Upper 2,ez-bar curls,17.5,15


In [73]:
df_routines = df_routines.set_index("name")
df_routines

,exercise,reps
name,,
Wave Loading 321 Lower 1,trap bar deadlift,3
Wave Loading 321 Lower 1,trap bar deadlift,2
Wave Loading 321 Lower 1,trap bar deadlift,1
Wave Loading 321 Lower 1,trap bar deadlift,3
Wave Loading 321 Lower 1,trap bar deadlift,2
...,...,...
Upper Lower 3 Sets Upper 2,press ups,20
Upper Lower 3 Sets Upper 2,press ups,20
Upper Lower 3 Sets Upper 2,face pulls,12


In [74]:
for workout_date in workout_dates:
    routine = df_workouts.loc[workout_date, "workoutroutine"].unique()[0]
    valid_exercises = set(df_routines.loc[routine, "exercise"].tolist())
    actual_exercises = set(df_workouts.loc[workout_date, "exercise"].tolist())
    combined = actual_exercises.union(valid_exercises)
    if len(combined) > len(valid_exercises):
        print("Bad exercise!")
        print(workout_date)